In [23]:
import json,re,os,datetime,time,string
from pathlib import Path
import requests
import langchain
from langchain.llms.base import LLM
from typing import Any, List, Optional,Dict, Mapping, Union
from deepeval.models.base_model import DeepEvalBaseLLM
import collections
import traceback,random
import pandas as pd
import evalplus.sanitize,evalplus.syncheck

In [ ]:
INFINI_API = "sk-c7cssl4bkglsrwf2"
INFINI_API_2 = "sk-c7erk6qaqhkz5t72"
INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]

In [ ]:
modelList = {
    'llama-3-70b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-70b-Instruct是此系列里，700亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-70b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年12月。",
    'llama-3-8b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-8b-Instruct是此系列里，80亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-8b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年3月。",
    'chatglm3':"ChatGLM3是智谱AI与清华KEG实验室发布的闭源模型，支持 8K 上下文，经过海量中英标识符的预训练与人类偏好对齐训练，相比一代模型在 MMLU、C-Eval、GSM8K 分别取得了16%、36%、280%的提升，并登顶中文任务榜单C-Eval。适用于对知识量、推理能力、创造力要求较高的场景，比如广告文案、小说写作、知识类写作、代码生成等。",
    'chatglm2-6b':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上，在 MMLU、C-Eval、GSM8K、BBH等主流学术数据集上，都得到了显著的性能提升，并通过基于 FlashAttention 技术，将对话模型的上下文长度（Context Length）提升至 8k tokens，允许更多轮次的对话。",
    'chatglm2-6b-32k':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。相较于ChatGLM2-6B，ChatGLM2-6b-32k支持更长的模型上下文至32k tokens。",
    'infini-megrez-7b':"由无问芯穹公司自主研发的70亿参数大语言模型。在逻辑推理、对话能力等方面有优秀的性能表现。配合无问芯穹自研高效推理引擎，同时支持Nvidia和AMD的GPU，具备更快的推理速度，在性能表现方面更上一层楼。",
    'llama-2-7b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-13b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-70b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-chat是其中700亿参数的大模型，适用于chat场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'llama-2-70b':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-base是其中700亿参数的基础大模型，适用于通用语言任务场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'baichuan2-7b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-7b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持4k tokens上下文。",
    'baichuan2-13b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持8k tokens上下文。",
    'baichuan2-13b-base':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-base是130亿参数规模的基础模型，适用于通用对话和文本续写，较chat模型更适合于复杂场景的微调后使用。该基模型支持4k tokens上下文。",
    'chatglm3-6b':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。ChatGLM3采用了全新设计的 Prompt 格式，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。模型支持 8k tokens上下文。",
    'chatglm3-6b-32k':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。相较于ChatGLM之前系列的模型，ChatGLM3采用了更多样的训练数据，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。ChatGLM3-6b-32k在ChatGLM3-6b 基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32k tokens长度的上下文。",
    'chatglm3-6b-base':"ChatGLM3-6b-base 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源的基础模型。ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。基础模型更适合于复杂场景的微调后使用，该基模型支持32k tokens上下文。",
    'qwen-7b-chat':"通义千问-7B-chat（Qwen-7B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的70亿参数规模的大语言模型。相较于Qwen-7B-Base模型，Qwen-7B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-14b-chat':"通义千问-14B-chat（Qwen-14B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的140亿参数规模的大语言模型。相较于Qwen-14B-Base模型，Qwen-14B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-72b-chat':"通义千问-72B-chat（Qwen-72B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的720亿参数规模的大语言模型。相较于Qwen-72B-Base模型，Qwen-72B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 32k tokens上下文。",
    'qwen-72b':"通义千问-72B（Qwen-72B）是阿里云研发的通义千问大模型系列的720亿参数规模的模型。Qwen-72B是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。模型支持 32k tokens上下文。",
    'qwen1.5-7b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-7b-chat是其中专用于chat场景的70亿参数的主流大小模型。",
    'qwen1.5-14b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-14b-chat是其中专用于chat场景的140亿参数的主流大小模型。",
    'qwen1.5-72b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-chat是其中专用于chat场景的720亿参数的大模型。",
    'qwen1.5-72b':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-base是其中的720亿参数的基础大模型，适合多种场景的使用。",
}
answerModelList = [    
    'llama-3-8b-instruct',
    # 'chatglm3',
    # 'chatglm2-6b',
    # 'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    # 'chatglm3-6b',
    # 'chatglm3-6b-32k',
    # 'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
]

In [ ]:
def LLMCompletions(prompt,modelName:str = "infini-megrez-7b",INFINI_API = "sk-c7cssl4bkglsrwf2",returnContent:bool = True,**kwargs):
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": "string",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+INFINI_API,
    } 
    response = requests.post(url, json=payload, headers=headers,)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        data = response.json()
        content = data['choices'][0]['message']['content']
        
        content = content.replace(',\n}','\n}')
        if returnContent:
            return content
        try:
            content = json.loads(content)
        except:
            content = content.replace('\n','')
        data['choices'][0]['message']['content'] = content
        
        return json.dumps(data['choices'][0]['message']['content'])

    else:
        print(response.status_code)
        return "Cannot connect to the model "+modelName

In [ ]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'time':-1
}
def joinErrorToData(errorFile,saveFile):
    with open(errorFile, 'r') as ef:
        data_ef = json.load(ef)
    fileName = data_ef['fileName']
    with open(saveFile) as sf:
        data_sf = json.load(sf)
    if fileName != data_sf['fileName']:
        print('FileName not match')
        return 
    if not data_ef['data']:
        print('The Errors of this ErrorFile all have been solved!')
        return 
    errorItem = []
    for item in data_ef['data']:
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItem.append(item)
            print("[error]:\t"+str(errorItem[-1]))
            continue
        print(actual_output)
        data_format_ins['time'] = delta
        data_format_ins['actual_output'] = actual_output
        data_sf['data'].append(data_format_ins.copy())
    data_ef['data'] = errorItem
    with open(saveFile,'w') as saveF:
        json.dump(data_sf,saveF)
    with open(errorFile,'w') as error:
        json.dump(data_ef,error)
    if errorItem:
        print("There are still some errors! ")
    else:
        print('The Errors of this ErrorFile all have been solved!')

In [ ]:
def removeDeprecatedModel(filePath):
    with open(filePath) as f:
        data = json.load(f)
    new_data = []
    for item in data['data']:
        if item['AnswerModel']  in answerModelList:
            new_data.append(item)
    data['data'] = new_data[:]
    with open(filePath,'w') as f:
        json.dump(data,f)

In [ ]:
# removeDeprecatedModel('./data/1-2_1_low_freq_ent_sample.json')

In [ ]:
# joinErrorToData('./data/4-2_r_with_triples_sampleError.json','./data/4-2_r_with_triples_sample.json')

In [ ]:
# diff = collections.Counter()

In [ ]:
# qapairs = []
# model_input_id = []
# with open('./data/1-1_2_high_freq_ent_sample.json') as saveF:
#     data = json.load(saveF)
# with open('E:/Repository/KoLA/Sample_Data/1-1_2_high_freq_ent_sample.json') as file:
#     data_o = json.load(file)
# instructions = data_o['adapter_spec']['instructions']
# for item in data_o['request_states']:
#     qapairs.append((item['instance']['id'],instructions+'\n'+item['instance']['input']['text']))
# for item in data['data']:
#     model_input_id.append([item['AnswerModel'],item['input'],item['id']])
# check = [ False for i in range(len(model_input_id))]
# for model in answerModelList:
#     for id,text in qapairs:
#         for i,item in enumerate(model_input_id):
#             if model == item[0] and text == item[1] and id == item[2]:
#                 check[i] = True

In [ ]:
# for model,text,id in model_input_id:
#     diff[model] += 1
#     diff[text] += 1
#     diff[id] += 1

In [ ]:
### deprecated ChatLLM,CustomLLM
# class ChatLLM(LLM):
#     modelName = "qwen1.5-72b-chat"
#     INFINI_API = "sk-c7cssl4bkglsrwf2"
#     temperature = 0.8
#     top_p = 1
#     top_k = -1
#     n = 1
#     max_tokens:int = None
#     stop:Optional[List[str]] = None
#     presence_penalty = 0
#     frequency_penalty = 0
    
#     headers = {
#             'Content-Type': "application/json",
#             'Accept': "*/*",
#             'Authorization': "Bearer "+INFINI_API,
#     }   
#     @property
#     def _llm_type(self)->str:
#         return "ChatLLM"
#     @property
#     def _identifying_params(self)->Mapping[str,Any]:
#         _param_dict = {
#             "modelName":ChatLLM().modelName,
#             "INFINI_API":ChatLLM().INFINI_API,
#             "stream":bool(ChatLLM().stream),
#             "temperature":ChatLLM().temperature,
#             "top_p":ChatLLM().top_p,
#             "top_k":ChatLLM().top_k,
#             "n":ChatLLM().n,
#             "max_tokens":ChatLLM().max_tokens,
#             "stop":ChatLLM().stop,
#             "presence_penalty":ChatLLM().presence_penalty,
#             "frequency_penalty":ChatLLM().frequency_penalty,
#         }
#         return _param_dict
#     @classmethod  
#     def _call(self, prompt: str, stop: Optional[List[str]]= None,  **kwargs: Any) -> str:
#         url = "https://cloud.infini-ai.com/maas/"+ChatLLM().modelName+"/nvidia/chat/completions"
#         payload = {
#             "model": "string",
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ],
#             "stream": False,
#             "temperature": kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature,
#             "top_p": kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p,
#             "top_k": kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k,
#             "n": kwargs['n'] if 'n' in kwargs else ChatLLM().n,
#             "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens,
#             "stop": stop if stop else ChatLLM().stop,
#             "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty,
#             "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
#         }
#         response = requests.post(url, json=payload, headers=ChatLLM().headers)
#         if response.status_code == 200:
#             response.encoding = 'utf-8'
#             data = response.json()
#             content = data['choices'][0]['message']['content']
#             if isinstance(content,str):    
#                 content = content.replace(',\n}','\n}')
#                 content = content.replace(']\n}',']}')
#                 content = content.replace('\\','\\\\')
#                 flag = False
#                 if 'statements' in content:
#                     regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
#                     flag = True
#                 elif 'verdicts' in content:
#                     regex = re.compile('\"verdicts\":\s+\[.*\]\}', re.DOTALL) 
#                     flag = True
#                 if flag:
#                     matchStr =regex.search(content)
#                     if matchStr:
#                         content = '{'+matchStr.group()
#             try:
#                 content = json.loads(content)
#             except:
#                 pass
#             data['choices'][0]['message']['content'] = content
                
#             return json.dumps(data['choices'][0]['message']['content'])

#         else:
#             return "Cannot connect to the model "+ChatLLM().modelName
#     def setParameter(self,**kwargs):
#         self.temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
#         self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else ChatLLM().top_p
#         self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else ChatLLM().top_k
#         self.n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
#         self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens
#         self.stop = kwargs['stop'] if 'stop' in kwargs else ChatLLM().stop
#         self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
#         self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty

# class CustomLLM(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model
#     ):
#         self.model = model

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         return chat_model.invoke(prompt)
#     async def a_generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         res = await chat_model.ainvoke(prompt)
#         return res

#     def get_model_name(self):
#         return "CustomLLM"

# custom_model = ChatLLM()

# evaluateModel = CustomLLM(model=custom_model)

In [ ]:
path = []

In [ ]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0
        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                print("response json success")
                content = data['choices'][0]['message']['content']
                if isinstance(content,str):   
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr:
                            content = '{'+matchStr.group()
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr is not None:
                            content ='{' +matchStr.group()
                            regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
                            matchStr = regex.findall(content)
                            for string in matchStr:
                                tmp = string.strip()[1:-1].replace('"','\\\"')
                                tmp = '\"'+tmp+"\""
                                content = content.replace(string,tmp)
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    

class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
    def load_model(self):
        return self.model
    def generate(self, prompt: str) -> str:
        global path
        chat_model = self.load_model()
        ret = chat_model.invoke(prompt)
        idx = 0
        if ret == "Cannot connect to the model "+self.get_model_name() and idx<2:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
custom_model = ChatLLM()
evaluateModel = CustomLLM(model=custom_model)

In [ ]:
content = "\"verdicts\": [{\"verdict\":\"yes\"} , {\"verdict\":\"no\",\"reason\":\"123\"}]"
regex = re.compile("\{\s*\"verdict\"\s*:.*?\}")
matchStr = regex.findall(content)
matchStr

In [ ]:
metrics_format_example = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format_example = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format_example
    ]
}

        

In [ ]:
for dirName,subDirName ,fileNames in os.walk('./data'):
    print(fileNames)

In [ ]:
fileNames = ['1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric,HallucinationMetric,BaseMetric
from deepeval.test_case import LLMTestCase

In [ ]:
metrics  = [
    AnswerRelevancyMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    ),
    HallucinationMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    )
]


In [ ]:
# with open('./eval/error/2-1_COPEN++csj_sample.json') as f:
#     data = json.load(f)
# data['data'][0]

In [ ]:
# test_case = LLMTestCase(input=data['data'][0]['input'],
#                         actual_output=data['data'][0]['actual_output'],
#                         context=[data['data'][0]['expected_output']])

In [ ]:
# metrics[0].measure(test_case)

In [ ]:
def evaluate_file(filename:Union[str,Path],save_file:Union[str,Path],error_file:Union[str,Path],force_save:bool = False,metrics:List[BaseMetric] = [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)]):
    """_summary_
        the function is used to evaluate the LLM output saved in `filename` by the metric in `metrics`,the successful eval results will be saved into `save_file` and the error item will be saved into `error_file`
    
    Args:
        `filename` (Union[str,Path]): the filename saves the LLM generation results
        `save_file` (Union[str,Path]): the filename will save the evaluate results
        `error_file` (Union[str,Path]): the filename will save the error eval item
        `force_save` (bool, optional): if the value is `True`,function will rerun all eval item in `filename` and directly override the `save_file` and `error_file`. 
                    Defaults to False.
        `metrics` (List[BaseMetric], optional): a list of evaluation metrics. 
                    Defaults to [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)].
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['input'] == item['input'] and x['actual_output'] == item['actual_output'] and x['AnswerModel'] == item['AnswerModel'] and x['expected_output'] == item['expected_output']:
            return True
        return False
    with open(filename,'r') as f:
        data = json.load(f)
    save,error = dict(),dict()
    if  Path(save_file).is_file() and not force_save:
        with open(save_file) as f:
            save = json.load(f)
        if 'fileName' in save and  save['fileName'] != data['fileName']:
            print("The save_file does not match the file name!")
            return
    else:
        save = {'fileName':data['fileName'],'class':data['class'],'data':[]}

    if Path(error_file).is_file() and not force_save:
        with open(error_file) as f:
            error = json.load(f)
        if 'fileName' in error and  error['fileName'] != data['fileName']:
            print("The error_file does not match the file name!")
            return
    else:
        error = {'fileName' :data['fileName'],'class':data['class'],'data':[]}
    for metric in metrics:
        for item in data['data']:
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[]
            }
            metrics_format_ins = {
                'metric_metadata':{
                    'metric':None,
                    'success':True,
                    'score':0.8,
                    'reason':'',
                    'statements':'',
                    'verdicts':'',
                    'evaluationCost': 0
                },
                'metric_configuration': {
                    'threshold': 0.5,
                    'evaluation_model': 'CustomLLM',
                    'strict_mode': False,
                    'include_reason': True
                }
            }
            errors_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[
                    {            
                        'metric_metadata':{
                            'metric':None,
                        },
                        'metric_configuration': {
                            'threshold': 0.5,
                            'evaluation_model': 'CustomLLM',
                            'strict_mode': False,
                            'include_reason': True
                        }
                    }
                ]
            }
            
            data_format_ins['id'] = item['id']
            data_format_ins['AnswerModel'] = item['AnswerModel']
            data_format_ins['input'] = item['input']
            data_format_ins['actual_output'] = item['actual_output']
            data_format_ins['expected_output'] = item['expected_output']
            tag = False
            for x in save['data']:
                if is_same_eval_item(item,x):
                    for metric_data in x['cached_metrics_data']:
                        if metric_data['metric_metadata']['metric'] == metric.__name__:
                            if metric_data['metric_configuration']['threshold'] == metric.threshold and metric_data['metric_configuration']['evaluation_model'] == metric.evaluation_model and metric_data['metric_configuration']['strict_mode'] == metric.strict_mode and metric_data['metric_configuration']['include_reason'] == metric.include_reason:
                                tag = True
                                print("HAVE:")
                                print(x)
                                break
            if tag:
                continue
            test_case = LLMTestCase(
                input= item['input'],
                actual_output=item['actual_output'],
                context=[item['expected_output']],
            )
            try:
                metric.measure(test_case)
            except Exception as e:
                print(e)
                path.append(e)
                errors_format_ins['id'] = item['id']
                errors_format_ins['AnswerModel'] = item['AnswerModel']
                errors_format_ins['input'] = item['input']
                errors_format_ins['actual_output'] = item['actual_output']
                errors_format_ins['expected_output'] = item['expected_output']
                errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
                errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
                error['data'].append(errors_format_ins.copy())
                print(errors_format_ins)
                with open(error_file,'w') as f:
                    json.dump(error,f,indent=4)
                continue
            metrics_format_ins['metric_metadata']['metric'] = metric.__name__
            metrics_format_ins['metric_metadata']['score'] = metric.score
            metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
            metrics_format_ins['metric_metadata']['reason'] = metric.reason
            
            metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
            metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
            metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

            metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
            metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
            metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
            metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

            data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
            flag = True
            for each in save['data']:
                if is_same_eval_item(each, data_format_ins):
                    each['cached_metrics_data'].append(metrics_format_ins.copy())
                    flag = False
                    break
            if flag:
                save['data'].append(data_format_ins.copy())
                print(data_format_ins)
            with open(save_file,'w') as f:
                json.dump(save,f,indent=4)

In [ ]:
def joinEvalErrorToData(errorFile:Union[str,Path],saveFile:Union[str,Path])->None:
    """_summary_
        the function is used to rerun the error item in the `errorFile` and append the results into the `saveFile` 
        
    Args:
        `errorFile` (Union[str,Path]): the JSON file saves the error item in the before running
        `saveFile` (Union[str,Path]): the JSON file saves the pass result
        
    Returns:
        None: the result will override the original file 
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['input'] == item['input'] and x['actual_output'] == item['actual_output'] and x['AnswerModel'] == item['AnswerModel'] and x['expected_output'] == item['expected_output']:
            return True
        return False
    with open(errorFile,'r') as f:
        data_er = json.load(f)
    with open(saveFile,'r') as f:
        data_sv = json.load(f)
    if data_er['fileName'] != data_sv['fileName']:
        print("The save_file does not match the error_file!")
        return
    error = {'fileName' :data_er['fileName'],'class':data_er['class'],'data':[]}
    while data_er['data']:
        item = data_er['data'].pop()
        data_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'context':None,
            'retrieval_context':None,
            'cached_metrics_data':[]
        }
        metrics_format_ins = {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        }
        errors_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'retrieval_context':None,
            'cached_metrics_data':[
                {            
                    'metric_metadata':{
                        'metric':None,
                    },
                    'metric_configuration': {
                        'threshold': 0.5,
                        'evaluation_model': 'CustomLLM',
                        'strict_mode': False,
                        'include_reason': True
                    }
                }
            ]
        }
        data_format_ins['id'] = item['id']
        data_format_ins['AnswerModel'] = item['AnswerModel']
        data_format_ins['input'] = item['input']
        data_format_ins['actual_output'] = item['actual_output']
        data_format_ins['expected_output'] = item['expected_output']
        
        tag = False
        for x in data_sv['data']:
            if is_same_eval_item(item,x):
                for metric_data in x['cached_metrics_data']:
                    if metric_data['metric_metadata']['metric'] == item['cached_metrics_data'][0]['metric_metadata']['metric']:
                        if metric_data['metric_configuration']['threshold'] == item['cached_metrics_data'][0]['metric_configuration']['threshold'] and metric_data['metric_configuration']['evaluation_model'] == item['cached_metrics_data'][0]['metric_configuration']['evaluation_model'] and metric_data['metric_configuration']['strict_mode'] == item['cached_metrics_data'][0]['metric_configuration']['strict_mode'] and metric_data['metric_configuration']['include_reason'] == item['cached_metrics_data'][0]['metric_configuration']['include_reason']:
                            tag = True
                            print("HAVE:")
                            print(x)
                            break
        if tag:
            continue
        test_case = LLMTestCase(
            input= item['input'],
            actual_output=item['actual_output'],
            context=[item['expected_output']],
        )
        if item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Answer Relevancy':
            metric = AnswerRelevancyMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        elif item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Hallucination':
            metric = HallucinationMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        else:
            print("unkonwn metric!")
            continue
        
        try:
            metric.measure(test_case)
        except Exception as e:
            print(traceback.print_exc())
            errors_format_ins['id'] = item['id']
            errors_format_ins['AnswerModel'] = item['AnswerModel']
            errors_format_ins['input'] = item['input']
            errors_format_ins['actual_output'] = item['actual_output']
            errors_format_ins['expected_output'] = item['expected_output']
            errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
            errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
            error['data'].append(errors_format_ins.copy())
            print(errors_format_ins)
            with open(errorFile,'w') as f:
                json.dump(data_er,f,indent=4)
            continue
        metrics_format_ins['metric_metadata']['metric'] = metric.__name__
        metrics_format_ins['metric_metadata']['score'] = metric.score
        metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
        metrics_format_ins['metric_metadata']['reason'] = metric.reason
        
        metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
        metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
        metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

        metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
        metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
        metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
        metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

        data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
        flag = True
        for each in data_sv['data']:
            if is_same_eval_item(each, data_format_ins):
                each['cached_metrics_data'].append(metrics_format_ins.copy())
                flag = False
                break
        if flag:
            data_sv['data'].append(data_format_ins.copy())
            print(data_format_ins)
        with open(saveFile,'w') as f:
            json.dump(data_sv,f,indent=4)
    
    with open(errorFile,'w') as f:
        json.dump(error,f,indent=4)

In [ ]:
with open('./eval/error/2-5_DocRED_sample.json') as f:
    tmp = json.load(f)
tmp

In [ ]:
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','r') as f:
#     data = json.load(f)
# for item in data['data']:
#     item['actual_output'] = item['actual_output'].replace("[/INST]",'')
#     item['actual_output'] = item['actual_output'].replace("[INST]",'')
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','w') as f:
#     json.dump(data,f,indent=4)

In [50]:
files_error = ['2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_with_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in files_error:
    joinEvalErrorToData('./eval/error/'+file,'./eval/save/'+file)

504

504

504

504

None
{'id': 'with_triples_sample_0', 'AnswerModel': 'baichuan2-13b-chat', 'input': 'Complete the following generated texts and make sure to contain all the events provided.\n### \n#### Known Events\n##### relation: uncovered\n Speaker: Alston & Bird\n Message: no facts to show that U . S . Soccer knew of the 1992 Incident when it hired Mr . Berhalter \n Receiver: Gregg Berhalter\n##### relation: harm\n Agent: Gregg Berhalter\n Cause: others \n Location: United States\n##### relation: injured\n Agent: Gregg Berhalter\n Cause: Rosalind\n Location: the University of North Carolina \n##### relation: preparing\n Agent: The next coach \n Content: the U . S . team for the 2026 World Cup \n Upcoming activity: World Cup\n Location: United States\n##### relation: guided\n Agent: Berhalter\n Patient: United States\n Method: men ’ s national team coach \n##### relation: knew\n Agent: USSF\n Patient: the 1992 Incident \n Location: United States\n##### relation: show\n Speaker: Alston & Bird\n Messa

Traceback (most recent call last):
  File "d:\Anaconda\envs\vllm\lib\site-packages\deepeval\metrics\utils.py", line 57, in trimAndLoadJson
    return json.loads(jsonStr)
  File "d:\Anaconda\envs\vllm\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "d:\Anaconda\envs\vllm\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\Anaconda\envs\vllm\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\游锦亮\AppData\Local\Temp\ipykernel_42008\194931906.py", line 104, in joinEvalErrorToData
    metric.measure(test_case)
  File "d:\Anaconda\envs\vllm\lib\site-packages\deepeval\metrics\answer_relevancy\answer_relevancy.py", line 60, in measure
    loop.run_u

In [ ]:

joinEvalErrorToData('./eval/error/2-5_DocRED_sample.json','./eval/save/2-5_DocRED_sample.json')

In [ ]:
path[0].text

In [ ]:
regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
matchStr = regex.findall(path[1])
# matchStr = regex.sub(lambda x:"\"reason\":\""+x.group(0).strip().replace("\"reason\":","").strip()[1:-1].replace('"','\\\"'),path[1])
matchStr

In [ ]:
matchStr[2].strip()[1:-1].replace('"','\\\"')

In [ ]:
regex.sub()

In [ ]:
path[1]

In [ ]:
['{"verdicts": [\n        {\n            "verdict": "idk"\n        },\n        {\n            "verdict": "no",\n            "reason": "The statement \'I am unable to provide an answer to your question\' acknowledges the inability to answer but doesn\'t provide any relevant information about the similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'it is not clear what "Rose" refers to\' is relevant as it points out the need for clarification to determine similarity."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please provide more context or information?\' is relevant as it directly asks for additional details to establish the similarity with Rose."\n        }\n    ]}',
 '{"verdicts": [\n        {\n            "verdict": "no",\n            "reason": "The statement \'I\'m sorry,\' does not provide any information or similarity comparison with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'I need more context to answer your question.\' is relevant as it acknowledges the need for additional information to determine similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please clarify what "Rose" refers to?\' is relevant as it seeks to understand the specific context or entity named Rose for a proper comparison."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'What you are looking for in terms of a conceptually similar entity?\' is relevant as it asks for the criteria or aspects to consider when identifying a conceptually similar entity to Rose."\n        }\n    ]}']

In [ ]:
def has_answer_kilt(answers, text) -> bool:
    text = normalize_kilt(text)
    for single_answer in answers:
        single_answer = normalize_kilt(single_answer)
        if single_answer in text:
            return True
    return False


# answer normalization
def normalize_kilt(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
# for file in fileNames:
#     evaluate_file('./data/'+file,'./eval/save/'+file,'./eval/error/'+file)

In [ ]:
with open('./eval/save/2-1_COPEN++csj_sample.json') as f:
    data_view = json.load(f)
noEvalItem = []
for item in data_view['data']:
    if len(item['cached_metrics_data']) <2:
        noEvalItem.append(item)
noEvalItem

In [ ]:
item = noEvalItem[0]

test_case = LLMTestCase(
    input= item['input'],
    actual_output=item['actual_output'],
    context=[item['expected_output']],
)
metric = AnswerRelevancyMetric(
    threshold=0.5,
    model = evaluateModel,
    include_reason=True,
)
metric.measure(test_case)


In [ ]:
for item in [noEvalItem[0]]:
    metrics_format_ins = {
        'metric_metadata':{
            'metric':None,
            'success':True,
            'score':0.8,
            'reason':'',
            'statements':'',
            'verdicts':'',
            'evaluationCost': 0
        },
        'metric_configuration': {
            'threshold': 0.5,
            'evaluation_model': 'CustomLLM',
            'strict_mode': False,
            'include_reason': True
        }
    }
    test_case = LLMTestCase(
        input= item['input'],
        actual_output=item['actual_output'],
        context=[item['expected_output']],
    )
    metric = AnswerRelevancyMetric(
        threshold=0.5,
        model = evaluateModel,
        include_reason=True,async_mode=False
    )
    metric.measure(test_case)
    
    metrics_format_ins['metric_metadata']['metric'] = metric.__name__
    metrics_format_ins['metric_metadata']['score'] = metric.score
    metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
    metrics_format_ins['metric_metadata']['reason'] = metric.reason
    
    metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
    metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
    metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

    metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
    metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
    metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
    metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason
    
    print(metrics_format_ins)

### MATH PROCESS AND EVAL

In [ ]:
# with open('./data/math401/math50.json') as f:
#     data = json.load(f)
# data_new  = {'fileName': data['fileName'],'data':[]}
# for item in data['data']:
#     data_format = {'id':item['id'],'input':item['input'],'expected_output':item['expected_output']}
#     data_new['data'].append(data_format.copy())
# with open('./data/math401/math50.json','w') as f:
#     json.dump(data_new,f,indent=4)

In [ ]:
with open('./data/math401/math50.json','r') as f:
    data  = json.load(f)
data['data']

In [ ]:
# with open('./data/math401/save/infini-megrez-7b.json','r') as f:
#     data_401 = json.load(f)
#     data_401_new = {'fileName': data_401['fileName'],'data':[]}
#     for item in data_401['data']:
#         for x in data['data']:
#             if item['input'] =='Only return the correct answer of the question.\n'+ x['input']:
#                 item['id'] = x['id']
#                 item['is_correct'] = -1
#                 data_401_new['data'].append(item.copy())
# with open('./data/math401/save/infini-megrez-7b.json','w') as f:
#     json.dump(data_401_new,f,indent=4)

In [ ]:
def sort_result(filePath:Union[str,Path])->None:
    """
    _summary_
        the function is used to sort the results saved in the JSON `filepath` by `id` field
        
    Args:
        `filePath` (Union[str,Path]): the JSON file path

    Returns:
        None: the result will override the original `filePath` 
    """
    with open(filePath,'r') as f:
        data = json.load(f)
    data['data'].sort(key = lambda x:x['id'])
    with open(filePath,'w') as f:
        json.dump(data,f,indent=4)


In [ ]:
def addField(file:Union[str,Path],field:str,default:Any)->None:
    """
    _summary_
        the function is used to add a field into the `.json` `file`

    Args:
        `file` (Union[str,Path]): the filename ,the file should be JSON file
        `field` (str): the field name
        `default` (Any): the default value of the added field,the value will add or subtract a Random from -500_000_000 to 500_000_000 if the field is `time`
        
    Returns:
        None: the result will override the original `file` 
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if field not in item:
            item[field] = default
            if field == 'time':
                item[field] += random.randint(-500_000_000,500_000_000)
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        addField(os.path.join(dir,file),'extract_answer',None)

In [ ]:
def getNumberAnswer(text:str)->Union[float,None]:
    """_summary_
        the function is used to extract the number from the text

    Args:
        `text` (str): the text contains the number
        
    Returns:
        Union[float,None]: the number in the `text`, or None if the `text` does not contain the number
    """
    text = text.split('=')
    if text:
        text = text[-1]
        regex = re.compile('([+-]?\d+[,0-9]*[.]?[0-9]*)')
        ret = regex.findall(text)
        if ret:
            return eval(ret[-1].replace(',',''))
        else:
            return None

In [ ]:
def processAnswer(file:Union[str,Path]):
    """
    _summary_:
        the function is used to process the answer that the LLM returned , namely extracting the number from the `actual_output` and save as `extract_answer`, and transform `expected_output` from str to number[int,float] 

    Args:
        `file` (Union[str,Path]): the filename of the file saving the answer created by the LLM
        
    Returns:
        None : the result will be written into the original `file`
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if item['is_correct'] == -1:
            item['extract_answer'] = getNumberAnswer(item['actual_output'])
            item['expected_output'] = eval(item['expected_output'])
            if item['extract_answer'] is None :
                item['is_correct'] = 0
            else:
                item['is_correct'] =1 if abs(item['expected_output']-item['extract_answer']) < 1e-3 else 0
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        processAnswer(os.path.join(dir,file))

In [ ]:
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         with open(os.path.join(dir,file),'r') as f:
#             data = json.load(f)
#         for item in data['data']:

#             if item['is_correct']:
#                 item['is_correct'] = 1
#             else:
#                 item['is_correct'] = 0
#         with open(os.path.join(dir,file),'w') as f:
#             json.dump(data,f,indent = 4)

In [8]:

def calculate_accuracy(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the accuracy of the model answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,accuracy),else only return accuracy `[0,1]`. 
                    Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and accuracy in format `(modelName:str,accuracy:float)`,else only return the accuracy in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    correct = 0
    for item in data['data']:
        correct += item['is_correct']
    accuracy =  correct/len(data['data'])
    if return_tuple:
        return (data['model'],accuracy)
    return accuracy

def calculate_nan_ratio(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the no number ratio of the LLM answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,nan_ratio),else only return nan_ratio `[0,1]`. Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and nan_ratio in format `(modelName:str,nan_ratio:float)`,else only return the nan_ratio in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    nan = 0
    for item in data['data']:
        if item['extract_answer'] is None:
            nan += 1
    nan_ratio = nan/len(data['data'])
    if return_tuple:
        return (data['model'],nan_ratio)
    return nan_ratio

In [17]:
def RE(y_true,y_pred):
    return min(10,abs(y_true-y_pred)/max(abs(y_true),1))
def calculate_RE(file:Union[str,Path],return_tuple:bool = False)->Union[List[float],tuple[str,List[float]]]:
    with open(file,'r') as f:
        data = json.load(f)
    RE_list = []
    for item in data['data']:
        ret = 0
        y_true = item['expected_output']
        y_pred = item['extract_answer']
        if y_pred is None:
            ret = 10
        else:
            ret = RE(y_true,y_pred)
        RE_list.append(ret)
    if return_tuple:
        return (data['model'],RE_list)
    return RE_list

In [21]:
# data_list = {'eval':'math','eval_dataset':'math50.json','data':[]}
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
#         nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
#         RE_List = calculate_RE(os.path.join(dir,file))
#         data_list['data'].append({'model':tmp[0],'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]})
# with open('./data/math401/math50_eval_result.json','w') as f:
#     json.dump(data_list,f,indent=4)

In [19]:
# data = {}
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
#         nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
#         RE_List = calculate_RE(os.path.join(dir,file))
#         data[tmp[0]] = {'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]}
# df = pd.DataFrame(data).T.sort_index()
# df.to_json('./data/math401/math50_eval_result_df.json')